# <p style="text-align: center; font-size: 30px;">📊 <b>Statistical ML: Group 10 - XRay Project</b> 📊</p>


## Libraries

In [27]:
# Libraries
import os
import csv
import xml.etree.ElementTree as ET
import pandas as pd

## Setting-Up & Loading the Data

In [33]:
# Set the environment variable for the VS Code session
os.environ['XRAY_DATA_PATH'] = '/Users/pranavpai/Code/StatsML/Statisitcal_Machine_Learning_Project/XRay'
# Use the XRAY_DATA_PATH environment variable
base_path = os.getenv('XRAY_DATA_PATH')

In [ ]:
# Function to convert XML to CSV
def xml_to_csv(xml_folder, output_csv):
    if os.path.exists(output_csv):
        print(f"{output_csv} already exists. Skipping conversion.")
        return
    
    with open(output_csv, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        # Write the header row with all the relevant properties
        csvwriter.writerow(['filename', 'width', 'height', 'depth', 'class', 'pose', 'truncated', 'difficult', 'xmin', 'ymin', 'xmax', 'ymax'])
        
        # Iterate over all XML files in the folder
        for xml_file in os.listdir(xml_folder):
            if xml_file.endswith('.xml'):
                tree = ET.parse(os.path.join(xml_folder, xml_file))
                root = tree.getroot()

                # Extract filename, width, height, and depth
                filename = root.find('filename').text
                size = root.find('size')
                width = int(size.find('width').text)
                height = int(size.find('height').text)
                depth = int(size.find('depth').text)

                # Iterate over all object elements in the XML
                for obj in root.findall('object'):
                    label = obj.find('name').text
                    pose = obj.find('pose').text
                    truncated = int(obj.find('truncated').text)
                    difficult = int(obj.find('difficult').text)
                    bbox = obj.find('bndbox')
                    xmin = int(bbox.find('xmin').text)
                    ymin = int(bbox.find('ymin').text)
                    xmax = int(bbox.find('xmax').text)
                    ymax = int(bbox.find('ymax').text)

                    # Write data to CSV
                    csvwriter.writerow([filename, width, height, depth, label, pose, truncated, difficult, xmin, ymin, xmax, ymax])

    print(f"{output_csv} has been created successfully.")

if base_path is None:
    raise EnvironmentError("Environment variable 'XRAY_DATA_PATH' is not set.")

# Construct paths using the base_path from the environment variable
train_xml_folder = os.path.join(base_path, 'Train', 'Annotations')
test_xml_folder = os.path.join(base_path, 'Test', 'Annotations')

# Output CSV paths
train_output_csv = os.path.join(base_path, 'Train', 'train_annotations.csv')
test_output_csv = os.path.join(base_path, 'Test', 'test_annotations.csv')

# Convert XML to CSV for train and test datasets
xml_to_csv(train_xml_folder, train_output_csv)
xml_to_csv(test_xml_folder, test_output_csv)


In [ ]:
# Load and display stats for CSV
def compute_csv_stats(csv_path, name):
    try:
        # Load the CSV into a DataFrame
        df = pd.read_csv(csv_path)

        # Display basic stats
        print(f"--- Stats for {name} ---")
        print(f"Total Rows: {len(df)}")
        print(f"Unique Images: {df['filename'].nunique()}")

        # Show a summary of the dataset (describe numeric fields)
        print("\nSummary Statistics:")
        print(df.describe())

        # Show the first 5 rows as a sample
        print("\nSample Data (first 5 rows):")
        print(df.head())

        # Add a blank line after each section for readability
        print("\n" * 3)

    except Exception as e:
        print(f"An error occurred while processing {name}: {e}")

compute_csv_stats(train_output_csv, 'Training Annotations')
compute_csv_stats(test_output_csv, 'Testing Annotations')